## 10장 인공신경망

### (1) 분류 예측 실습

- 변수 지정 및 전처리

In [1]:
#1. 모듈 및 함수 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE


#2. 변수 지정(독립변수/종속변수)
df = pd.read_csv('Ashopping.csv', encoding = 'cp949')
X = df[['총매출액', '구매금액대', '1회 평균매출액','거래기간', '평균 구매주기']]
Y = df['할인민감여부']

#3. 데이터 분할(학습용/평가용 데이터 세트)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

#4. 전처리(표준화, 원핫 인코딩) 
ct = ColumnTransformer([('scaling', StandardScaler(), ['총매출액', '1회 평균매출액', '거래기간', 
'평균 구매주기']), ('onehot', OneHotEncoder(sparse = False),['구매금액대'])])
ct.fit(X_train)
X_train=ct.transform(X_train)
X_test=ct.transform(X_test)

#5. 오버 샘플링
smote = SMOTE(random_state=0)
X_train, Y_train = smote.fit_sample(X_train, Y_train)


Using TensorFlow backend.


- 모형 학습 및 예측

In [2]:
#1. 모듈 및 함수 불러오기
from sklearn.neural_network import MLPClassifier

#2. 모형 생성 
model = MLPClassifier(random_state=0, alpha = 0.001, hidden_layer_sizes = [50] )

#3. 모형 학습 및 예측 
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print('평가용 데이터 세트에 대한 예측값\n', Y_pred)


평가용 데이터 세트에 대한 예측값
 [1 1 0 0 1 1 0 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 1 1 0 0 0 0 1 1 1 1 1 1 1
 0 1 0 0 1 1 1 0 0 1 0 0 0 1 0 0 0 1 1 0 1 1 1 0 1 1 0 0 0 1 1 0 0 1 0 0 0
 1 0 0 0 0 1 0 0 0 1 1 1 0 0 1 0 1 0 0 0 0 1 1 0 0 0 1 1 0 1 0 1 1 0 1 1 0
 1 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 0 0 1 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 1 0 0 1 1 0 1 0 1 0 0 1 0 0 0 1 0 1
 0 0 1 1 1 0 1 0 1 1 1 1 0 1 0 0 1 0 1 1 0 1 1 1 1 0 0 0 1 0 1 0 0 0 1 1 1
 1 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 1 1 0 0
 0 1 1 1 0 0 0 0 0 1 0 1 1 0 0 1 1 0 0 0 0 1 1 0 1 0 0 1 0 0 0 1 0 1 1 0 0
 0 0 0 0]


C:\Users\user\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


- 모형 평가

In [3]:
print('학습용 데이터 세트 정확도: {:.3f}'.format(model.score(X_train, Y_train)))
print('평가용 데이터 세트 정확도: {:.3f}'.format(model.score(X_test, Y_test)))

학습용 데이터 세트 정확도: 0.898
평가용 데이터 세트 정확도: 0.887


In [4]:
#1. 모듈 및 함수 불러오기
from sklearn.metrics import classification_report  

#2. 정밀도, 재현율, F1 스코어 출력
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.92      0.89      0.90       177
           1       0.84      0.89      0.87       123

    accuracy                           0.89       300
   macro avg       0.88      0.89      0.88       300
weighted avg       0.89      0.89      0.89       300



### (2) 수치 예측 실습

- 변수 지정 및 전처리

In [5]:
#1. 모듈 및 함수 불러오기

import numpy as np

#2. 변수 지정 및 
X=df[df.이탈여부==0][['방문빈도', '총 할인 금액', '고객등급', '구매유형', '거래기간', '할인민감여부','평균 구매주기']]
Y =np.log1p(df[df.이탈여부==0]['1회 평균매출액'])

#3. 데이터 분할
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

#4. 표준화 및 원핫인코딩
ct = ColumnTransformer([('scaling', StandardScaler(), ['방문빈도', '총 할인 금액', '거래기간', 
'평균 구매주기']), ('onehot', OneHotEncoder(sparse = False),['고객등급', '할인민감여부', 
'구매유형'])])
ct.fit(X_train)
X_train=ct.transform(X_train)
X_test=ct.transform(X_test)


- 모형 학습 및 평가

In [6]:
#1. 모듈 및 함수 불러오기
from sklearn.neural_network import MLPRegressor

#2. 모형 생성 
model = MLPRegressor(random_state=0,  alpha =1, max_iter= 1000, hidden_layer_sizes = [50, 50])

#3. 모형 학습 및 예측
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print('평가용 데이터 세트에 대한 예측값\n', Y_pred)

평가용 데이터 세트에 대한 예측값
 [12.6477617  12.08440792 11.91145523 13.3082498  11.61975833 12.21617464
 11.09956832 12.42788763 11.88221406 12.34200744 12.37716096 12.39688378
 12.35676987 12.09788225 12.79515645 12.06765255 12.73711254 12.86210825
 12.03530154 12.38181906 12.75071175 11.57998222 12.31781106 12.61832592
 11.10738745 12.98650446 12.66696291 12.24779974 12.30830584 12.01606842
 12.70721122 12.97291006 12.51729587 13.1232342  12.08939231 12.02779396
 12.06544449 13.01804133 11.96701783 13.32169349 12.75257899 12.02382266
 12.51066148 12.05052917 12.35385259 12.56924187 11.31787287 12.11968403
 12.70654195 12.38737966 11.99907342 13.29785469 11.36649102 12.88950884
 12.7930371  12.42713632 12.51557994 12.46126645 12.17941176 12.12676422
 12.86684542 13.09886034 11.50126616 12.42901966 12.08592068 11.72087048
 11.97876738 11.67683805 12.99883331 12.33187645 12.35060844 11.66813714
 11.97609295 11.75543921 12.41210675 13.04049095 12.62930439 12.99376303
 12.1672865  12.42885985 12.431

C:\Users\user\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


- 모형 평가

In [7]:
print('학습용 데이터 세트 결정계수: {:.3f}'.format(model.score(X_train, Y_train)))
print('평가용 데이터 세트 결정계수: {:.3f}'.format(model.score(X_test, Y_test)))


학습용 데이터 세트 결정계수: 0.730
평가용 데이터 세트 결정계수: 0.609


In [8]:
#1. 모듈 및 함수 불러오기
from sklearn.metrics import mean_squared_error
from math import sqrt

#2. RMSE 계산
rmse= sqrt(mean_squared_error(Y_test, Y_pred))
print('RMSE:{:.3f}'.format(rmse))

RMSE:0.406
